# 📊 01 - Data Exploration
## Volcano Monitoring Framework - Initial Data Analysis

## Introduction
This notebook provides an introduction to the Volcano Monitoring Framework data structure and demonstrates basic data exploration techniques.

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
import json
import yaml

# Add src to path
sys.path.append('../src')

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("🌋 Volcano Monitoring Framework - Data Exploration")

## 1. Project Structure Overview

In [ ]:
import os

def list_project_structure():
    """Display project structure."""
    structure = {}
    for root, dirs, files in os.walk('..'):
        if '.git' in root or '__pycache__' in root:
            continue
        level = root.replace('..', '').count(os.sep)
        if level <= 2:  # Show only up to 2 levels deep
            indent = '  ' * level
            print(f'{indent}📁 {os.path.basename(root) or "volcano"}/')
            if level < 2:
                subindent = '  ' * (level + 1)
                for file in sorted(files)[:3]:
                    if file.endswith(('.py', '.md', '.yaml', '.txt')):
                        print(f'{subindent}📄 {file}')
                if len(files) > 3:
                    print(f'{subindent}... and {len(files)-3} more files')

list_project_structure()

## 2. Generate Demo Data

In [ ]:
def generate_seismic_data(days=30, events_per_day=5):
    """Generate synthetic seismic data."""
    np.random.seed(42)
    
    dates = pd.date_range('2026-01-01', periods=days, freq='D')
    data = []
    
    for date in dates:
        for _ in range(np.random.poisson(events_per_day)):
            event = {
                'time': date + pd.Timedelta(hours=np.random.uniform(0, 23.9)),
                'magnitude': np.random.uniform(1.0, 4.0),
                'depth': np.random.uniform(1.0, 15.0),
                'latitude': 37.75 + np.random.uniform(-0.05, 0.05),
                'longitude': 14.99 + np.random.uniform(-0.05, 0.05)
            }
            data.append(event)
    
    return pd.DataFrame(data)

def generate_gps_data(days=30):
    """Generate synthetic GPS deformation data."""
    np.random.seed(42)
    
    dates = pd.date_range('2026-01-01', periods=days, freq='D')
    
    # Simulate inflation trend
    time = np.arange(days)
    inflation_trend = 0.1 * time  # mm/day trend
    
    data = {
        'time': dates,
        'easting': inflation_trend + np.random.normal(0, 0.5, days),
        'northing': 0.8 * inflation_trend + np.random.normal(0, 0.5, days),
        'vertical': 0.5 * inflation_trend + np.random.normal(0, 0.3, days)
    }
    
    return pd.DataFrame(data)

# Generate data
seismic_df = generate_seismic_data(60)
gps_df = generate_gps_data(60)

print(f"✅ Generated {len(seismic_df)} seismic events")
print(f"✅ Generated {len(gps_df)} GPS measurements")

## 3. Data Visualization

In [ ]:
# Create figure with subplots
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Seismic Data Analysis', fontsize=16, fontweight='bold')

# 1. Magnitude distribution
axes[0, 0].hist(seismic_df['magnitude'], bins=15, edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Magnitude')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Magnitude Distribution')
axes[0, 0].axvline(seismic_df['magnitude'].mean(), color='red', linestyle='--', label=f'Mean: {seismic_df["magnitude"].mean():.2f}')
axes[0, 0].legend()

# 2. Depth distribution
axes[0, 1].hist(seismic_df['depth'], bins=15, edgecolor='black', alpha=0.7, color='orange')
axes[0, 1].set_xlabel('Depth (km)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Depth Distribution')

# 3. Time series of event count
seismic_df['date'] = seismic_df['time'].dt.date
daily_counts = seismic_df.groupby('date').size()
axes[0, 2].plot(daily_counts.index, daily_counts.values, marker='o', linestyle='-', linewidth=2)
axes[0, 2].set_xlabel('Date')
axes[0, 2].set_ylabel('Events per day')
axes[0, 2].set_title('Daily Seismic Activity')
axes[0, 2].tick_params(axis='x', rotation=45)

# 4. Magnitude vs Depth
scatter = axes[1, 0].scatter(seismic_df['depth'], seismic_df['magnitude'], 
                            c=seismic_df['magnitude'], cmap='viridis', alpha=0.6)
axes[1, 0].set_xlabel('Depth (km)')
axes[1, 0].set_ylabel('Magnitude')
axes[1, 0].set_title('Magnitude vs Depth')
plt.colorbar(scatter, ax=axes[1, 0], label='Magnitude')

# 5. Spatial distribution
spatial_scatter = axes[1, 1].scatter(seismic_df['longitude'], seismic_df['latitude'], 
                                   c=seismic_df['depth'], cmap='coolwarm', 
                                   s=seismic_df['magnitude']*20, alpha=0.6)
axes[1, 1].set_xlabel('Longitude')
axes[1, 1].set_ylabel('Latitude')
axes[1, 1].set_title('Spatial Distribution (size=magnitude, color=depth)')
plt.colorbar(spatial_scatter, ax=axes[1, 1], label='Depth (km)')

# 6. Cumulative events
cumulative_events = seismic_df.sort_values('time').reset_index()
cumulative_events['cumulative'] = range(1, len(cumulative_events) + 1)
axes[1, 2].plot(cumulative_events['time'], cumulative_events['cumulative'], 
               linewidth=2, color='green')
axes[1, 2].set_xlabel('Time')
axes[1, 2].set_ylabel('Cumulative Events')
axes[1, 2].set_title('Cumulative Seismic Events')
axes[1, 2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# GPS Data Visualization
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('GPS Deformation Analysis', fontsize=16, fontweight='bold')

# 1. Easting time series
axes[0, 0].plot(gps_df['time'], gps_df['easting'], marker='o', linewidth=2)
axes[0, 0].set_xlabel('Time')
axes[0, 0].set_ylabel('Easting (mm)')
axes[0, 0].set_title('Horizontal Easting Deformation')
axes[0, 0].tick_params(axis='x', rotation=45)

# 2. Northing time series
axes[0, 1].plot(gps_df['time'], gps_df['northing'], marker='o', linewidth=2, color='orange')
axes[0, 1].set_xlabel('Time')
axes[0, 1].set_ylabel('Northing (mm)')
axes[0, 1].set_title('Horizontal Northing Deformation')
axes[0, 1].tick_params(axis='x', rotation=45)

# 3. Vertical time series
axes[1, 0].plot(gps_df['time'], gps_df['vertical'], marker='o', linewidth=2, color='green')
axes[1, 0].set_xlabel('Time')
axes[1, 0].set_ylabel('Vertical (mm)')
axes[1, 0].set_title('Vertical Deformation')
axes[1, 0].tick_params(axis='x', rotation=45)

# 4. 3D deformation path
from mpl_toolkits.mplot3d import Axes3D
ax_3d = fig.add_subplot(2, 2, 4, projection='3d')

# Plot deformation path
ax_3d.plot(gps_df['easting'], gps_df['northing'], gps_df['vertical'], 
          linewidth=2, marker='o', markersize=4)
ax_3d.set_xlabel('Easting (mm)')
ax_3d.set_ylabel('Northing (mm)')
ax_3d.set_zlabel('Vertical (mm)')
ax_3d.set_title('3D Deformation Path')

# Mark start and end
ax_3d.scatter(gps_df['easting'].iloc[0], gps_df['northing'].iloc[0], gps_df['vertical'].iloc[0], 
             color='green', s=100, label='Start', marker='s')
ax_3d.scatter(gps_df['easting'].iloc[-1], gps_df['northing'].iloc[-1], gps_df['vertical'].iloc[-1], 
             color='red', s=100, label='End', marker='^')
ax_3d.legend()

plt.tight_layout()
plt.show()

## 4. Statistical Analysis

In [ ]:
def calculate_b_value(magnitudes, mc=1.5):
    """Calculate b-value using maximum likelihood estimation."""
    mags_above = magnitudes[magnitudes >= mc]
    if len(mags_above) < 10:
        return None, None
    
    b_value = np.log10(np.exp(1)) / (np.mean(mags_above) - mc)
    std_error = b_value / np.sqrt(len(mags_above))
    return b_value, std_error

# Calculate b-value over time
seismic_df['day'] = (seismic_df['time'] - seismic_df['time'].min()).dt.days
b_values = []
days = []

for day in range(0, 60, 5):  # Every 5 days
    day_data = seismic_df[seismic_df['day'].between(day, day+5)]
    if len(day_data) >= 10:
        b, std = calculate_b_value(day_data['magnitude'].values)
        if b is not None:
            b_values.append(b)
            days.append(day + 2.5)  # Middle of window

# Plot b-value evolution
fig, ax = plt.subplots(figsize=(10, 6))
ax.errorbar(days, b_values, yerr=[bv*0.1 for bv in b_values], 
           fmt='o-', linewidth=2, capsize=5, capthick=2)
ax.axhline(y=1.0, color='red', linestyle='--', alpha=0.5, label='Typical b-value = 1.0')
ax.set_xlabel('Time (days from start)')
ax.set_ylabel('b-value')
ax.set_title('Temporal Evolution of b-value (indicator of stress state)')
ax.legend()
ax.grid(True, alpha=0.3)

# Interpretation
print("📈 b-value Interpretation:")
print("• b-value > 1.0: More small earthquakes (stress release)")
print("• b-value < 1.0: More large earthquakes (stress accumulation)")
print("• Decreasing trend: Increasing stress (potential unrest)")

plt.show()

## 5. Deformation Analysis

In [ ]:
# Calculate deformation velocities
gps_df['time_numeric'] = (gps_df['time'] - gps_df['time'].min()).dt.total_seconds() / 86400

# Linear regression for velocity calculation
def calculate_velocity(times, values):
    """Calculate linear velocity."""
    A = np.vstack([times, np.ones_like(times)]).T
    m, c = np.linalg.lstsq(A, values, rcond=None)[0]
    return m, c  # m = velocity (mm/day), c = intercept

# Calculate velocities for each component
east_vel, east_int = calculate_velocity(gps_df['time_numeric'], gps_df['easting'])
north_vel, north_int = calculate_velocity(gps_df['time_numeric'], gps_df['northing'])
vert_vel, vert_int = calculate_velocity(gps_df['time_numeric'], gps_df['vertical'])

# Calculate horizontal velocity magnitude
horizontal_vel = np.sqrt(east_vel**2 + north_vel**2)

# Display results
print("🚀 Deformation Velocities:")
print(f"   Easting: {east_vel:.3f} mm/day")
print(f"   Northing: {north_vel:.3f} mm/day")
print(f"   Horizontal magnitude: {horizontal_vel:.3f} mm/day")
print(f"   Vertical: {vert_vel:.3f} mm/day")
print(f"   Total 3D: {np.sqrt(horizontal_vel**2 + vert_vel**2):.3f} mm/day")

# Interpretation
print("\n📊 Interpretation:")
if horizontal_vel > 5.0:
    print("   ⚠️  High deformation rate - potential unrest")
elif horizontal_vel > 2.0:
    print("   📈 Moderate deformation - increased monitoring recommended")
else:
    print("   ✅ Normal background deformation")

# Plot with trend lines
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

components = [('easting', east_vel, east_int, 'Easting'),
              ('northing', north_vel, north_int, 'Northing'),
              ('vertical', vert_vel, vert_int, 'Vertical')]

colors = ['blue', 'orange', 'green']

for idx, (comp, vel, intercept, title) in enumerate(components):
    ax = axes[idx // 2, idx % 2]
    
    # Plot data
    ax.scatter(gps_df['time'], gps_df[comp], alpha=0.6, label='Measurements')
    
    # Plot trend line
    trend = vel * gps_df['time_numeric'] + intercept
    ax.plot(gps_df['time'], trend, 'r-', linewidth=3, 
            label=f'Trend: {vel:.3f} mm/day')
    
    ax.set_xlabel('Time')
    ax.set_ylabel(f'{title} (mm)')
    ax.set_title(f'{title} Deformation')
    ax.legend()
    ax.tick_params(axis='x', rotation=45)
    ax.grid(True, alpha=0.3)

# Plot vector plot in bottom right
ax_vec = axes[1, 1]
ax_vec.quiver(np.zeros(len(gps_df)), np.zeros(len(gps_df)),
             gps_df['easting'], gps_df['northing'], 
             gps_df['vertical'], scale=1, cmap='viridis', alpha=0.6)
ax_vec.set_xlabel('Easting (mm)')
ax_vec.set_ylabel('Northing (mm)')
ax_vec.set_title('3D Deformation Vectors (color=vertical)')
ax_vec.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Summary and Next Steps

In [ ]:
# Summary statistics
print("📊 SUMMARY STATISTICS")
print("=" * 50)

print("\n🌋 SEISMIC DATA:")
print(f"   Total events: {len(seismic_df)}")
print(f"   Date range: {seismic_df['time'].min().date()} to {seismic_df['time'].max().date()}")
print(f"   Magnitude range: {seismic_df['magnitude'].min():.2f} to {seismic_df['magnitude'].max():.2f}")
print(f"   Average magnitude: {seismic_df['magnitude'].mean():.2f}")
print(f"   Average depth: {seismic_df['depth'].mean():.2f} km")

print("\n📍 SPATIAL EXTENT:")
print(f"   Latitude: {seismic_df['latitude'].min():.4f}° to {seismic_df['latitude'].max():.4f}°")
print(f"   Longitude: {seismic_df['longitude'].min():.4f}° to {seismic_df['longitude'].max():.4f}°")

print("\n📈 GPS DEFORMATION:")
print(f"   Total measurements: {len(gps_df)}")
print(f"   Total easting change: {gps_df['easting'].iloc[-1] - gps_df['easting'].iloc[0]:.2f} mm")
print(f"   Total northing change: {gps_df['northing'].iloc[-1] - gps_df['northing'].iloc[0]:.2f} mm")
print(f"   Total vertical change: {gps_df['vertical'].iloc[-1] - gps_df['vertical'].iloc[0]:.2f} mm")

# Calculate daily event rate
total_days = (seismic_df['time'].max() - seismic_df['time'].min()).days
if total_days > 0:
    daily_rate = len(seismic_df) / total_days
    print(f"\n📅 ACTIVITY RATE:")
    print(f"   Average events per day: {daily_rate:.2f}")

# Save data for later use
print("\n💾 SAVING DATA:")
os.makedirs('../data/processed/exploration', exist_ok=True)
seismic_df.to_csv('../data/processed/exploration/seismic_demo.csv', index=False)
gps_df.to_csv('../data/processed/exploration/gps_demo.csv', index=False)
print("   Saved: ../data/processed/exploration/seismic_demo.csv")
print("   Saved: ../data/processed/exploration/gps_demo.csv")

print("\n✅ Data exploration complete!")

## Next Steps

1. **Notebook 02**: Parameter index calculation
2. **Notebook 03**: Physics-based modeling (Mogi source)
3. **Notebook 04**: Chaos theory analysis (Lyapunov exponents)
4. **Notebook 05**: Multi-parameter integration framework
5. **Notebook 06**: Validation and performance metrics
6. **Notebook 07**: Case studies analysis